In [47]:
import math
import cv2
import numpy as np
from time import time
import mediapipe as mp
from pygame import mixer
import tempfile
from gtts import gTTS
import requests


In [48]:
mp_pose = mp.solutions.pose
pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
url = "http://192.168.0.224:80"


In [49]:
# 計算角度的副函式
def calculateAngle(landmark1, landmark2, landmark3):
    # 獲取所需座標
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3
    # 計算三點之間的夾角
    angle = math.floor(math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2)))
    if angle < 0:
        angle += 360
    if angle>180:
        angle=360-angle
    return angle

In [50]:
# Text to Speech
def speak(sentence, lang):
    with tempfile.NamedTemporaryFile(delete=True) as fp:
        tts=gTTS(text=sentence, lang=lang)
        tts.save('{}.mp3'.format(fp.name))
        mixer.init()
        mixer.music.load('{}.mp3'.format(fp.name))
        mixer.music.play(1)

In [51]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    height, width, _ = image.shape
    landmarks = []
    if results.pose_landmarks:
            # 畫關鍵點在圖片上
        mp_drawing.draw_landmarks(image=image, landmark_list=results.pose_landmarks,
                    connections=mp_pose.POSE_CONNECTIONS)
        for landmark in results.pose_landmarks.landmark:
        # 將關鍵點加進list內.
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                    (landmark.z * width)))    
    try:
        l_elbow = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
    except:
        l_elbow=0
    try:
        # 取得右肩、右肘和右腕之間的夾角。    # 12 14 16 
        r_elbow = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])
    except:
        r_elbow=0
    try:
        # 取得左肘、左肩和左臀之間的夾角。    # 13 11 23
        l_shoulder = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
    except:
        l_shoulder=0
    try:
        # 取得右臀、右肩和右肘之間的夾角。    # 14 12 24
        r_shoulder = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
    except:
        r_shoulder=0
    try: # 左臀角度 
        l_hip = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value])       
    except:
        l_hip=0
    try: # 右臀角度
        r_hip = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value])           
    except:
        r_hip=0
    try:
        # 取得左臀、左膝蓋和左腳踝之間的角度。# 23 25 27
        l_knee = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
    except:
        l_knee=0        
    try:
        # 取得右臀、右膝蓋和右腳踝之間的角度。# 24 26 28 
        r_knee = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    except:
        r_knee=0
    
    return landmarks,image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee

In [52]:
#嬰兒式
def ep4_baby1():
    lastTime1 = time()
    lastTime2 = time()
    lastTime3 = time()
    lastTime4 = time()
    cap = cv2.VideoCapture(0)
    with mp_pose.Pose(static_image_mode=False, 
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5, 
                      model_complexity=2)as pose:
        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            # Make detection
            output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee=mediapipe_detection(frame, pose)
          
            condA = l_knee>50 or r_knee>50
            condB = l_shoulder<150 or r_shoulder<150    
            condC = l_hip>70 or r_hip>70

            if condA and (time()-lastTime1)<3:
                text1= 'hip back on heel'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                # lastTime = time()
            elif condA and (time()-lastTime1)>3 :
                text1= 'hip back on heel'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                lastTime1 = time()
                speak("臀往後坐在腳跟上",'zh-tw')
            else:    
                text1= ' '
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                if condB and (time()-lastTime2)<3:
                    text2= 'hands straight forward'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                elif condB and (time()-lastTime2)>3:
                    speak("雙手伸直往前延伸",'zh-tw')
                    text2= 'hands straight forward'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    lastTime2 = time()
                else:
                    text2= ' '
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    if condC and (time()-lastTime3)<3:
                        text3= 'hip back on heel 2'
                        cv2.putText(output_image, text3, (30, 90),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)    
                    elif condC and (time()-lastTime3)>3:
                        speak("臀往後坐在腳跟上",'zh-tw')
                        text3= 'hip back on heel 2'
                        cv2.putText(output_image, text3, (30, 90),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                        lastTime3 = time()
                    elif (time()-lastTime4)>3:
                        text3= ' success '
                        speak("完美!",'zh-tw')
                        cv2.putText(output_image, text3, (30, 90),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                        lastTime4 = time()
                        break


            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
        cap.release()
        cv2.destroyAllWindows()


In [53]:
#測試雙手伸直
def test_hand():
    lastTime1 = time()
    lastTime2 = time()
    cap = cv2.VideoCapture(0)
    with mp_pose.Pose(static_image_mode=False, 
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5, 
                      model_complexity=2)as pose:
        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee=mediapipe_detection(frame, pose)

            condA = l_elbow<140 
            condB = r_elbow<140   
             
            if condA and (time()-lastTime1)<3:
                text1= 'left hand stright'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            elif condA and (time()-lastTime1)>3 :
                text1= 'left hand stright'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                lastTime1 = time()
                speak("左手打平",'zh-tw')
            else:    
                text1= ' '
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                if condB and (time()-lastTime2)<3 :
                    text2= 'right hand straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                elif condB and (time()-lastTime2)>3 :
                    speak("右手打平",'zh-tw')
                    text2= 'right hand straight'
                    lastTime2 = time()
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                else:
                    text2= ' success '
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
        

            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
        cap.release()
        cv2.destroyAllWindows()


In [54]:
def hand_up():
    lastTime1 = time()
    lastTime2 = time()
    lastTime3 = time()
    cap = cv2.VideoCapture(0)
    with mp_pose.Pose(static_image_mode=False, 
                      min_detection_confidence=0.5,
                      min_tracking_confidence=0.5, 
                      model_complexity=2)as pose:
        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee=mediapipe_detection(frame, pose)

            condA = l_hip<165 or r_hip<165
            condB = l_knee<165 or r_knee<165
            condC = l_shoulder<165 or r_shoulder<165
             
            if condA and (time()-lastTime1)<3:
                text1= 'standing move'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            elif condA and (time()-lastTime1)>3 :
                text1= 'standing move'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                lastTime1 = time()
                speak("請保持站姿",'zh-tw')
            else:    
                text1= ' '
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                if condB and (time()-lastTime2)<3 :
                    text2= 'standing move'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                elif condB and (time()-lastTime2)>3 :
                    speak("請保持站姿",'zh-tw')
                    text2='standing move'
                    lastTime2 = time()
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                else:
                    text2= ' '
                    cv2.putText(output_image, text2, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    if condC and (time()-lastTime3)<3 :
                        text3= 'hands over head'
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    elif condB and (time()-lastTime3)>3 :
                        speak("雙手高舉過頭",'zh-tw')
                        text3='hands over head'
                        lastTime3 = time()
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    else:
                        text3= ' success '
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
        

            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
        cap.release()
        cv2.destroyAllWindows()

In [55]:
# pose 2 前彎向下
def pose2():
    lastTime1 = time()
    lastTime2 = time()
    try:
        response = requests.get(url, timeout=5) # 發送 GET 請求
        if response.status_code == 200:
            print("伺服器運作正常")
        else:
            print(f"伺服器回傳了錯誤的狀態碼: {response.status_code}")
    except requests.exceptions.ConnectionError:
        print(requests.exceptions.ConnectionError)
        
    cap = cv2.VideoCapture("yoga-1.mp4")  # 開啟鏡頭
    with mp_pose.Pose(static_image_mode=False, 
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5, 
                    model_complexity=0)as pose:

        # speak("待會要做前彎向下,請注意平衡安全__  ",'zh-tw')   
        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            landmarks,output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee=mediapipe_detection(frame, pose)

            condA =l_hip>40 or r_hip>40
            condB = l_knee<150 or r_knee<150
                
            if condA and (time()-lastTime1)<4:
                text1= 'upper body close to thigh'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            elif condA and (time()-lastTime1)>=4 :
                text1= 'upper body close to thigh'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                speak("上身貼近大腿",'zh-tw')
                lastTime1 = time()
                if(requests.exceptions.ConnectionError):
                    pass
                else:
                    data = {'shoulder': '1'}
                    response = requests.post(url, data=data)
                    print(response.text)
                
            else:    
                text1= ' '
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                if condB and (time()-lastTime2)<4 :
                    text2= 'keep your legs straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                elif condB and (time()-lastTime2)>=4 :
                    text2= 'keep your legs straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    speak("雙腿盡量伸直",'zh-tw')
                    lastTime2 = time() 
                    if(requests.exceptions.ConnectionError):
                        pass
                    else:
                        data = {'knee': '1'}
                        response = requests.post(url, data=data)
                        print(response.text)
                                       
                else:
                    text2= ' success '
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    return
            color=(0,0,255)
                    
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_elbow),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_elbow),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_shoulder),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_shoulder),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_hip),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_HIP.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_HIP.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_hip),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_knee),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_knee),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass



            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
        cap.release()
        cv2.destroyAllWindows()

In [56]:
# pose 3 背部向前延伸
def pose3():
    lastTime1 = time()
    lastTime2 = time()
    cap = cv2.VideoCapture("yoga-1.mp4")  # 開啟鏡頭
    with mp_pose.Pose(static_image_mode=False, 
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5, 
                    model_complexity=0)as pose:

        # speak("待會要做前彎向下,請注意平衡安全__  ",'zh-tw')   
        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            landmarks,output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee=mediapipe_detection(frame, pose)

            condA =l_hip<50 or r_hip<50
            condB = l_knee<160 or r_knee<160
                
            if condA and (time()-lastTime1)<4:
                text1= 'back forward'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            elif condA and (time()-lastTime1)>=4 :
                text1= 'back forward'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                speak("上身抬高,背部向前延伸",'zh-tw')
                data = {'shoulder': '1'}
                response = requests.post(url, data=data)
                print(response.text)
                lastTime1 = time()
            else:    
                text1= ' '
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                if condB and (time()-lastTime2)<4 :
                    text2= 'keep your legs straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                elif condB and (time()-lastTime2)>=4 :
                    text2= 'keep your legs straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    speak("雙腿盡量伸直",'zh-tw')
                    data = {'knee': '1'}
                    response = requests.post(url, data=data)
                    print(response.text)
                    lastTime2 = time() 
                else:
                    text2= ' success '
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            color=(0,0,255)
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_elbow),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_elbow),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_shoulder),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_shoulder),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_hip),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_HIP.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_HIP.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_hip),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_knee),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_knee),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass



            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
        cap.release()
        cv2.destroyAllWindows()

In [57]:
# pose 4 下犬式
def pose4():
    lastTime1 = time()
    lastTime2 = time()
    cap = cv2.VideoCapture("yoga-1.mp4")  # 開啟鏡頭
    with mp_pose.Pose(static_image_mode=False, 
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5, 
                    model_complexity=0)as pose:

        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            landmarks,output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee=mediapipe_detection(frame, pose)

            condA = l_knee<150 or r_knee<150
            condB = l_shoulder<150 or r_shoulder<150   
                    
            if condA and (time()-lastTime1)<4:
                text1= 'legs straight'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            elif condA and (time()-lastTime1)>=4 :
                text1= 'legs straight'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                speak("雙腳伸直",'zh-tw')
                data = {'knee': '1'}
                response = requests.post(url, data=data)
                print(response.text)
                lastTime1 = time()
            else:    
                text1= ' '
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                if condB and (time()-lastTime2)<4 :
                    text2= 'shoulder and back straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                elif condB and (time()-lastTime2)>=4 :
                    text2= 'shoulder and back straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    speak("肩膀與背部打直",'zh-tw')
                    data = {'shoulder': '1'}
                    response = requests.post(url, data=data)
                    print(response.text)
                    lastTime2 = time() 
                else:
                    text2= ' success '
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            color=(0,0,255)
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_elbow),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_elbow),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_shoulder),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_shoulder),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_hip),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_HIP.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_HIP.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_hip),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_knee),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_knee),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass



            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
        cap.release()
        cv2.destroyAllWindows()


In [58]:
# pose 5 平板式
def pose5():
    lastTime1 = time()
    lastTime2 = time()
    lastTime3 = time()

    cap = cv2.VideoCapture("yoga-1.mp4")  # 開啟鏡頭
    with mp_pose.Pose(static_image_mode=False, 
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5, 
                    model_complexity=0)as pose:

        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            landmarks,output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee=mediapipe_detection(frame, pose)

            condA = l_shoulder>100 or r_shoulder>100
            condB = l_knee <150 or r_knee <150  
            condC = l_hip <150 or r_hip <150  

            if condA and (time()-lastTime1)<4:
                text1= 'Shoulders come directly over wrists'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            elif condA and (time()-lastTime1)>=4 :
                text1= 'Shoulders come directly over wrists'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                speak("肩膀來到手腕的正上方",'zh-tw')
                data = {'shoulder': '1'}
                response = requests.post(url, data=data)
                print(response.text)
                lastTime1 = time()
            else:    
                text1= ' '
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                if condB and (time()-lastTime2)<4 :
                    text2= 'hip and leg straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                elif condB and (time()-lastTime2)>=4 :
                    text2= 'ship and leg straight'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    speak("膝蓋打直",'zh-tw')
                    data = {'knee': '1'}
                    response = requests.post(url, data=data)
                    print(response.text)
                    lastTime2 = time() 
                else:
                    text2= ' '
                    cv2.putText(output_image, text2, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    if condC and (time()-lastTime3)<4 :
                        text3= 'hip straight'
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    elif condC and (time()-lastTime3)>=4 :
                        text3= 'hip straight'
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                        speak("臀部打平",'zh-tw')
                        data = {'hip':'1'}
                        response = requests.post(url, data=data)
                        print(response.text)
                        lastTime3 = time() 
                    else:
                        text3= ' success '
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    

            color=(0,0,255)
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_elbow),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_elbow),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_shoulder),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_shoulder),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_hip),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_HIP.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_HIP.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_hip),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_knee),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_knee),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass



            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
        cap.release()
        cv2.destroyAllWindows()


In [59]:
# pose 6 眼鏡蛇式
def pose6():
    lastTime1 = time()
    lastTime2 = time()
    lastTime3 = time()

    cap = cv2.VideoCapture("yoga-1.mp4")  # 開啟鏡頭
    with mp_pose.Pose(static_image_mode=False, 
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5, 
                    model_complexity=0)as pose:

        while cap.isOpened():
            # Read feed
            ret, frame = cap.read()
            landmarks,output_image,l_elbow,r_elbow,l_shoulder,r_shoulder,l_hip,r_hip,l_knee,r_knee=mediapipe_detection(frame, pose)

            condA = l_knee<160 or r_knee<160
            condB = l_hip >150 or r_hip >150
            condC = l_shoulder>30 or r_shoulder>30

            if condA and (time()-lastTime1)<4:
                text1= 'straight the legs'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            elif condA and (time()-lastTime1)>=4 :
                text1= 'straight the legs'
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                speak("雙腿伸直",'zh-tw')
                data = {'knee':'1'}
                response = requests.post(url, data=data)
                print(response.text)
                lastTime1 = time()
            else:
                text1= ' '
                cv2.putText(output_image, text1, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                if condB and (time()-lastTime2)<4 :
                    text2= 'low-body on the floor'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                elif condB and (time()-lastTime2)>=4 :
                    text2= 'low-body on the floor'
                    cv2.putText(output_image, text2, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    speak("下半身貼好在地板上",'zh-tw')
                    data = {'hip':'1'}
                    response = requests.post(url, data=data)
                    print(response.text)
                    lastTime2= time() 
                else:    
                    text2= ' '
                    cv2.putText(output_image, text2, (30, 30),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    if condB and (time()-lastTime3)<4 :
                        text3= 'upper body improvement '
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                    elif condB and (time()-lastTime3)>=4 :
                        text3= 'upper body improvement '
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
                        speak("上半身提高",'zh-tw')
                        data = {'shoulder':'1'}
                        response = requests.post(url, data=data)
                        print(response.text)
                        lastTime3 = time() 
                    else:
                        text3= ' success '
                        cv2.putText(output_image, text3, (30, 60),cv2.FONT_HERSHEY_PLAIN, 1, (0,0,255), 2)
            color=(0,0,255)
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_elbow),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_elbow),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_shoulder),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_shoulder),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_hip),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_HIP.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_HIP.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_hip),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass
            # try:
            #     cv2.putText(
            #             output_image,
            #             str(l_knee),
            #             (
            #                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value][0],
            #                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value][1],
            #             ),
            #             cv2.FONT_HERSHEY_SIMPLEX,
            #             1,color,2,cv2.LINE_AA,)
            # except:
            #     pass
            try:
                cv2.putText(
                        output_image,
                        str(r_knee),
                        (
                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][0],
                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value][1],
                        ),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,color,2,cv2.LINE_AA,)
            except:
                pass



            cv2.imshow('OpenCV Feed', output_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
        cap.release()
        cv2.destroyAllWindows()


In [60]:
import pygame
pygame.init()

# 播放 MP3 音檔
pygame.mixer.music.load('EP18.mp3')
pygame.mixer.music.play()

## 開頭
# pygame.mixer.music.set_pos(380) # 6:20
# pygame.time.wait(27000) #26s
# pygame.mixer.music.pause()
## pose2
pygame.mixer.music.unpause()
pygame.mixer.music.set_pos(407) # 6:46
pygame.time.wait(3000) #34s
pygame.mixer.music.pause()
pose2()
## pose3
pygame.mixer.music.play()
pygame.mixer.music.set_pos(440) # 7:20
pygame.time.wait(9000) #9s
pygame.mixer.music.pause()
## pose4
# pygame.mixer.music.set_pos(449) # 7:29
# pygame.time.wait(43000) #43s
# pygame.mixer.music.pause()
## pose5
# pygame.mixer.music.set_pos(492) # 8:12
# pygame.time.wait(7000) #7s
# pygame.mixer.music.pause()
## 過渡
# pygame.mixer.music.set_pos(499) # 8:19
# pygame.time.wait(19500) #19.5s
# pygame.mixer.music.pause()
## pose6
# pygame.mixer.music.set_pos(518) # 7:29
# pygame.time.wait(16000) #10s
# pygame.mixer.music.pause()
## pose4
# pygame.mixer.music.set_pos(534) # 7:29
# pygame.time.wait(19000) #10s
# pygame.mixer.music.pause()
## pose2
# pygame.mixer.music.set_pos(553) # 7:29
# pygame.time.wait(20500) #10s
# pygame.mixer.music.pause()

# # 繼續播放 MP3 音檔
# pygame.mixer.music.unpause()

# # 停止 MP3 音檔
# pygame.mixer.music.stop()

    
pygame.quit()

<class 'requests.exceptions.ConnectionError'>
<class 'requests.exceptions.ConnectionError'>


KeyboardInterrupt: 